# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
import pandas as pd
import numpy as np
import json
import math
import string

from __future__ import division
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv('amazon_baby.csv', dtype=[('name',np.str_), ('review', np.str_),('rating', np.float)])

Now, let us see a preview of what the dataset looks like.

In [3]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products.iloc[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    if isinstance(text, basestring):
        import string
        return text.translate(None, string.punctuation) 
    else:
        return ''

In [6]:
products['review_clean'] = products['review'].apply(remove_punctuation)

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

C:\Anaconda2\envs\dato-env-2.7.9\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [11]:
idx_train, idx_test = json.load(open('module-2-assignment-train-idx.json')), json.load(open('module-2-assignment-test-idx.json'))
train_data, test_data = products.iloc[idx_train], products.iloc[idx_test]

print len(train_data)
print len(test_data)

133416
33336


## Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
Compute the occurrences of the words in each review and collect them into a row vector.
Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

In [12]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') # Use this token pattern to keep single-letter words
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [13]:
sentiment_model = LogisticRegression()
sentiment_model.fit(train_data['word_count'], train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as follows:

In [86]:
sentiment_weights = pd.DataFrame(columns=['word','coef'], index=vectorizer.vocabulary_)
sentiment_weights['word'] = vectorizer.vocabulary_
sentiment_weights['coef'] = sentiment_model.coef_.flatten()
sentiment_weights = sentiment_weights.sort('coef', ascending=False)
len(sentiment_weights)

121712

There are a total of `121712` coefficients in the model, plus interception. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0).

In [82]:
num_positive_weights = sum(sentiment_weights['coef'] >= 0)
num_negative_weights = sum(sentiment_weights['coef'] < 0)

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 86640 
Number of negative weights: 35072 


**Quiz question:** How many weights are >= 0?

In [17]:
print "Quiz question 1: %s " % num_positive_weights

Quiz question 1: 86640 


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [59]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: float64


,name,review,rating,review_clean,sentiment,prediction,word_count_subset
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1,1,"(0, 2)\t1\n (0, 18)\t1\n (1, 0)\t2\n (1, ..."
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1,-1,"(0, 2)\t1\n (0, 18)\t1\n (1, 0)\t2\n (1, ..."
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1,-1,"(0, 2)\t1\n (0, 18)\t1\n (1, 0)\t2\n (1, ..."


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [19]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [20]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [21]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60471249  -3.17516748 -10.42736019]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [22]:
print "Class predictions according to scores:"
print [1 if s > 0 else -1 for s in scores]

Class predictions according to scores:
[1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [23]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to GraphLab Create:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from sklearn.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [24]:
print "Class predictions according to scores:" 
print [1 / (1 + math.exp(-s)) for s in scores]

Class predictions according to scores:
[0.9963330176948809, 0.040110983027071355, 2.9610255204276893e-05]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [25]:
print "Class predictions according to GraphLab Create:"
predictions = [x[1] for x in sentiment_model.predict_proba(sample_test_matrix)]
print predictions

Class predictions according to GraphLab Create:
[0.99633301769488092, 0.040110983027071355, 2.9610255204276893e-05]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

In [26]:
print "Quiz question 2: %s " % (np.argmin(predictions) + 1)

Quiz question 2: 3 


# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use sklearn to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`.
2.  Sort the data according to those predictions and pick the top 20.

In [39]:
test_data['prediction'] = sentiment_model.predict(test_matrix)
test_data_sorted = test_data.sort('prediction', ascending=False)
test_data_top = test_data_sorted.head(20)

C:\Anaconda2\envs\dato-env-2.7.9\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [40]:
top_quiz = ('Snuza Portable Baby Movement Monitor', 'MamaDoo Kids Foldable Play Yard Mattress Topper, Blue', 'Britax Decathlon Convertible Car Seat, Tiffany', 'Safety 1st Exchangeable Tip 3 in 1 Thermometer')
print "Quiz question 3:"
print [n for n in test_data_top['name'] if n in top_quiz]

Quiz question 3:
[]


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [41]:
test_data_bottom = test_data_sorted.tail(20)

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

In [42]:
bottom_quiz = ('The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit', 'JP Lizzy Chocolate Ice Classic Tote Set', 'Peg-Perego Tatamia High Chair, White Latte', 'Safety 1st High-Def Digital Monitor')
print "Quiz question 4:"
print [n for n in test_data_bottom['name'] if n in bottom_quiz]

Quiz question 4:
[]


## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [43]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    pred = model.predict(data)

    # Compute the number of correctly classified examples
    correct = sum(pred == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = correct / data.shape[0]

    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [50]:
accuracy = get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])
print('Accuracy by sklearn %s' % accuracy_score(test_data['sentiment'], sentiment_model.predict(test_matrix)))

Accuracy by sklearn 0.932265418766


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

In [51]:
print "Quiz question 5: %s " % accuracy
print "Quiz question 6: No "

Quiz question 5: 0.932265418766 
Quiz question 6: No 


## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [52]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [53]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. Note that we are performing this on both the training and test set.

In [60]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words)
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [61]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [66]:
print get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])
print('Accuracy by sklearn %s' % accuracy_score(test_data['sentiment'], simple_model.predict(test_matrix_word_subset)))

0.869360451164
Accuracy by sklearn 0.869360451164


Now, we will inspect the weights (coefficients) of the **simple_model**:

In [125]:
simple_weights = pd.DataFrame(columns=['word','coef'])#, index=range(len(significant_words)))
simple_weights['word'] = significant_words
simple_weights['coef'] = simple_model.coef_.flatten()
simple_weights

,word,coef
0,love,1.363690
1,great,0.944000
2,easy,1.192538
3,old,0.085513
4,little,0.520186
5,perfect,1.509812
6,loves,1.673074
7,well,0.503760
8,able,0.190909
9,car,0.058855


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [128]:
simple_weights = simple_weights.sort('coef', ascending=False)
simple_weights

,word,coef
6,loves,1.673074
5,perfect,1.509812
0,love,1.363690
2,easy,1.192538
1,great,0.944000
4,little,0.520186
7,well,0.503760
8,able,0.190909
3,old,0.085513
9,car,0.058855


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [129]:
print "Quiz question 7: %s " % num_positive_weights

Quiz question 7: 86640 


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [154]:
positive_significant_words = [c[0] for c in simple_weights.values.tolist() if c[1] >= 0]
sentiment_positive_words = [c[0] for c in sentiment_weights.values.tolist() if c[1] >= 0]
if len([e for e in positive_significant_words if e not in sentiment_positive_words]) == 0:
    print 'Quiz question 8: Yes'
else:
    print 'Quiz question 8: No'

Quiz question 8: No


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [156]:
accuracy_sentiment_train = get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [158]:
accuracy_simple_train = get_classification_accuracy(simple_model, train_matrix_word_subset, train_data['sentiment'])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

In [159]:
if accuracy_sentiment_train > accuracy_simple_train:
    print 'Quiz question 9: sentiment_model'
else:
    print 'Quiz question 9: simple_model'

Quiz question 9: sentiment_model


Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [160]:
accuracy_sentiment_test = get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [161]:
accuracy_simple_test = get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

In [162]:
if accuracy_sentiment_test > accuracy_simple_test:
    print 'Quiz question 10: sentiment_model'
else:
    print 'Quiz question 10: simple_model'

Quiz question 10: sentiment_model


## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [163]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [164]:
accuracy_majority = num_positive / (num_positive + num_negative)
print "Quiz question 11: %s " % accuracy_majority

Quiz question 11: 0.840708760568 


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [165]:
if accuracy_sentiment_test > accuracy_majority:
    print 'Quiz question 12: Yes'
else:
    print 'Quiz question 12: No'

Quiz question 12: Yes
